# Day 08 answers

## 1

In [ ]:
def get_ice_data(year):
    #read in data but only first 5 rows since we know we need to chage the reading parameters
    #use f_format to change the values in the string 
    read_in_data=pd.read_table(datadirectory+f'gl_{year}_{year+1}_ice.csv',nrows=5)
    
    #get column names
    ice_names=read_in_data.iloc[2].str.split(" +", n = 8, expand = True)
    ice_names_dict=dict(zip(ice_names.keys(),ice_names.iloc[0,]))
    
    #make a clean table
    ice_clean=pd.read_csv(datadirectory+f'gl_{year}_{year+1}_ice.csv',skiprows=5,header=None,delim_whitespace=True)
    ice_clean.rename(columns=ice_names_dict,inplace=True)
    lake_initials={'Sup.':'SU','Mich.':'MI','Huron':'HU','Erie':'ER','Ont.':'ON'}
    ice_clean.rename(columns=lake_initials,inplace=True)
    ice_clean.drop(['St.Clr','GL Total'],axis=1,inplace=True)
    
    #add date column
    ice_clean['date']=pd.to_datetime(ice_clean['Year'].astype(str)+' '+ice_clean['Day'].astype(str), format='%Y %j')
    #add month column
    ice_clean['Month']=pd.DatetimeIndex(ice_clean['date']).month.astype(int)
    #drop date column
    ice_clean.drop(['date'],axis=1,inplace=True)

    #return the clean table
    return ice_clean

get_ice_data(2016)

## 2

In [ ]:
#1 Lake yearly ice_concentration% average and max
melt_ice.groupby(['Lake','Year'])['ice_concentration%'].agg(('mean', 'max')).reset_index()

#2. Lake monthly count and min per year
melt_ice.groupby(['Lake','Month'])['ice_concentration%'].agg(('count', 'min')).reset_index()

#3. Yearly standard_deviation for the entire region 
table1=pd.pivot_table(melt_ice, values=['ice_concentration%'], 
                      index=['Year'],aggfunc={'ice_concentration%': [np.std]})
table1.columns=table1.columns.droplevel(0)
table1=table1.reset_index()
table1

#4. Monthly ptp for the entire region in the past 10 years 
table2=pd.pivot_table(melt_ice, values=['ice_concentration%'], 
                      index=['Month'],aggfunc={'ice_concentration%': [np.ptp]})
table2.columns=table2.columns.droplevel(0)
table2=table2.reset_index()
table2

# 3

## part1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1) Read from URL
url = "https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv"
penguins = pd.read_csv(url)


# 2) quick look
display(penguins.head())
print("Shape:", penguins.shape)

# 3) Drop missing in key columns
needed = ['species','sex','body_mass_g','bill_length_mm','bill_depth_mm']
penguins_clean = penguins.dropna(subset=needed)

## part2

In [ ]:
# 4) Feature engineering with assign
penguins_clean = penguins_clean.assign(
    mass_kg = penguins_clean['body_mass_g'] / 1000,
    bill_ratio = penguins_clean['bill_length_mm'] / penguins_clean['bill_depth_mm']
)

# 5) Groupby summary
summary = (
    penguins_clean
    .groupby(['species','sex'], as_index=False)
    .agg(mean_mass_kg=('mass_kg','mean'),
         mean_bill_ratio=('bill_ratio','mean'))
    .sort_values('mean_mass_kg', ascending=False)
)
display(summary.head())

# 6) Pivot table: species x sex → mass_kg mean
pivot = penguins_clean.pivot_table(index='species', columns='sex', values='mass_kg', aggfunc='mean')
pivot

## part3

In [ ]:
# 7) Plot
ax = pivot.plot(kind='bar', figsize=(7,4))
ax.set_xlabel('Species'); ax.set_ylabel('Mean mass (kg)')
ax.set_title('Penguin mass by species and sex (mean)')
plt.tight_layout(); plt.show()

# 8) Save cleaned data
penguins_clean.to_csv('../output/penguins_clean.csv', index=False)
print("Saved penguins_clean.csv")